In [1]:
import pandas as pd
test_df = pd.read_csv('./Dataset/test.csv')
test_img_paths = test_df['img_path'].values

from collections import OrderedDict
df = pd.read_csv('./Dataset/train.csv')
classes = {}
for i in df['artist']:
    if i not in classes:
        # count 데이터 수
        classes[i] = 0
    else:
        classes[i] +=1
convert_labels = sorted(classes.items(), key=lambda x : x[1], reverse=True)
print(convert_labels)

for i in range(len(convert_labels)):
    classes[convert_labels[i][0]]=[i,convert_labels[i][1]]
print(classes)

[('Vincent van Gogh', 628), ('Edgar Degas', 488), ('Pablo Picasso', 302), ('Pierre-Auguste Renoir', 232), ('Albrecht Du rer', 219), ('Paul Gauguin', 219), ('Francisco Goya', 203), ('Rembrandt', 180), ('Titian', 172), ('Marc Chagall', 172), ('Alfred Sisley', 164), ('Paul Klee', 141), ('Rene Magritte', 136), ('Andy Warhol', 131), ('Amedeo Modigliani', 131), ('Henri Matisse', 120), ('Sandro Botticelli', 119), ('Mikhail Vrubel', 117), ('Hieronymus Bosch', 114), ('Leonardo da Vinci', 100), ('Salvador Dali', 98), ('Peter Paul Rubens', 96), ('Kazimir Malevich', 90), ('Pieter Bruegel', 84), ('Frida Kahlo', 83), ('Diego Velazquez', 80), ('Joan Miro', 75), ('Andrei Rublev', 73), ('Raphael', 72), ('Giotto di Bondone', 71), ('Gustav Klimt', 68), ('El Greco', 64), ('Jan van Eyck', 63), ('Camille Pissarro', 63), ('Edouard Manet', 61), ('Henri de Toulouse-Lautrec', 60), ('Vasiliy Kandinskiy', 59), ('Claude Monet', 58), ('Piet Mondrian', 58), ('Henri Rousseau', 51), ('Diego Rivera', 49), ('William Tur

In [2]:
new_df = pd.read_csv('./Dataset/artists_info.csv')
for name in classes.keys():
    for i in range(len(new_df)):
        if new_df.loc[i]['name'] == name:
            classes[name].extend(new_df.loc[i].iloc[1:])
            classes[name] = tuple(classes[name])
classes = OrderedDict(sorted(classes.items(), key = lambda t : t[1][1],reverse=True))

In [3]:
import cv2
from torch.utils.data import Dataset, DataLoader
import torch

class CustomDataset(Dataset):
    def __init__(self, img_paths, labels, class_info,transforms=None):
        self.img_paths = img_paths
        self.labels = labels
        self.transforms = transforms
        self.classes = class_info
    
    def __getitem__(self, index):
        img_path = './Dataset'+self.img_paths[index][1:]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']

        if self.labels is not None:
            label = torch.zeros([50], dtype=torch.float32)
            label[self.classes[self.labels[index]][0]] = 1
            # print(f'artist name {self.labels[index]} , label = {self.classes[self.labels[index]][0]}')
            return image, label
        else:
            return image
    def __len__(self):
        return len(self.img_paths)      
    
    def getclasses(self):
        return self.classes 

In [4]:
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

test_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=(0.485, 0.456, 0.406), std= (0.229,0.224,0.224), max_pixel_value=255.0, always_apply=False, p=1.0),
    ToTensorV2()
])

In [5]:
test_dataset = CustomDataset(test_img_paths, None,classes, test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

In [6]:
import torch
from tqdm import tqdm
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    
    model_preds = []
    
    with torch.no_grad():
        for img in tqdm(iter(test_loader)):
            img = img.float().to(device)
            
            model_pred = model(img)
            model_preds += model_pred.argmax(1).detach().cpu().numpy().tolist()
    
    print('Done.')
    return model_preds

In [7]:
import os
GPU_NUM = 1 # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device) # change allocation of current GPU
os.environ['CUDA_LAUNCH_BLOCKING']="1"
print(device)
print(f'torch version : {torch.__version__}')

cuda:1
torch version : 1.12.1


In [8]:
import torch.nn as nn
from torchvision import models
import timm
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(classes)):
        super(BaseModel, self).__init__()
        #self.backbone = convnext_large(weight=ConvNeXt_Large_Weights.DEFAULT)
        # self.backbone = models.convnext_base(weights=models.ConvNeXt_Base_Weights.DEFAULT)
        self.backbone = timm.models.regnetx_064(pretrained=True,num_classes=50)
        #self.backbone = timm.create_model('coatnet_3_224',pretrained=True)
        #self.backbone = models.convnext_small(weights=models.ConvNeXt_Small_Weights.IMAGENET1K_V1)
        #self.backbone= models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)
        #self.backbone = timm.create_model('vit_base_patch16_224', pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)
        self.drop = nn.Dropout(0.5,inplace=True)
    def forward(self, x):
        x = self.backbone(x)
        # x = self.drop(x)
        # x = self.classifier(x)
        return x
        
import torchsummary
model = BaseModel(num_classes=50)
torchsummary.summary(model, (3,224,224),device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
          Identity-2         [-1, 32, 112, 112]               0
              ReLU-3         [-1, 32, 112, 112]               0
    BatchNormAct2d-4         [-1, 32, 112, 112]              64
       ConvNormAct-5         [-1, 32, 112, 112]               0
            Conv2d-6        [-1, 168, 112, 112]           5,376
          Identity-7        [-1, 168, 112, 112]               0
              ReLU-8        [-1, 168, 112, 112]               0
    BatchNormAct2d-9        [-1, 168, 112, 112]             336
      ConvNormAct-10        [-1, 168, 112, 112]               0
           Conv2d-11          [-1, 168, 56, 56]          84,672
         Identity-12          [-1, 168, 56, 56]               0
             ReLU-13          [-1, 168, 56, 56]               0
   BatchNormAct2d-14          [-1, 168,

In [9]:
checkpoint = torch.load('./checkpoint/best_test.pth')
# print(checkpoint['model_state_dict'])
model.load_state_dict(checkpoint['model_state_dict'], strict=False)

<All keys matched successfully>

In [10]:
preds = inference(model, test_loader, device)

100%|██████████| 396/396 [01:45<00:00,  3.77it/s]

Done.


In [11]:
print(preds[:5])
result = []
for i in preds:

    result.append(convert_labels[i][0])
print(result[:5])
submit = pd.read_csv('./Dataset/sample_submission.csv')


[32, 11, 7, 12, 15]
['Jan van Eyck', 'Paul Klee', 'Rembrandt', 'Rene Magritte', 'Henri Matisse']


In [12]:
submit['artist']=result
submit.head()
submit.to_csv('./Dataset/submit.csv', index=False)